# Optimizing Financial Aid Resources

**Notebook by [Dev Nambi](https://blogs.uw.edu/fractals/)**


### Goals

This notebook uses the enrollment prediction model created elsewhere and optimizes the distribution of funds given the constraints involved (total scholarship funding, explainability).

* Run 


### Process

* A. Define constraints, such as the average SAT score, or average high school GPA, the different scholarship values, and the number of students to admit.
* B. Load in student data, software libraries, and model objects.
* C. Pass 1 - Arbitrary Predictions
* D. Calculate and visualize yield, net revenue, and scholarship info per segment.
* E. Pass 2 - Transfer Learning
* F. Calculate and visualize yield, net revenue, and scholarship info in an explainable way.
* G. Save off the results for later use.


### Resources

* https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/pareto-optimized-road-trip/optimized-state-capitols-trip.ipynb

## Part A - Define Goals and Constraints

Next we're going to have to specify constraints we care about:

* **Maximum Student Enrollment** - The maximum number of domestic non-resident students you would like to enroll
* **Average Student SAT** - The minimum average SAT for domestic non-resident students you're willing to accept
* **Award Amounts** - The different scholarship amounts to award.
* **Scholarship Fund Amount** - The amount of money to distribute via Purple and Gold scholarships

#### First, Define Constraints

In [1]:
fund_amount = 3000000   #3 million dollars

maximum_student_enrollment = 2100

#### Now, Define Goals

In [2]:
average_student_sat = 1090

award_amounts = [1000,2000,3000,4000,5000,6000,6500]

#### Add in useful variables

In [3]:
tuition_amount = 34143

useful_interaction_list = [51,133,126,73,59]

## Part B - Load in student data, software libraries, and model objects

### Part B1 - Load software libraries

In [6]:
# Load in software libraries
import os
import sys
import pandas as pd
import numpy as np
import pickle

# Get data-munging libraries
from sklearn import preprocessing
from sklearn import decomposition

# Get ML libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Get Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

### Part B2 - Load model objects

In [5]:
model_location = 'logit_model.pkl'
scalar_location = 'model_scalar.pkl'

with open(model_location, 'rb') as model_input:
    predict_model = pickle.load(model_input)

with open(scalar_location, 'rb') as scalar_input:
    model_scalar = pickle.load(scalar_input)

print (str(predict_model))
print (str(model_scalar))

ValueError: unsupported pickle protocol: 4

### Part B3 - Load student data

In [498]:
df = pd.read_csv('Transformed_Data_set.csv')
#df.describe()

### Part C - First Pass, Arbitrary Predictions

1. Get the data into a handy format
2. Predict students who will come with no PuGo
3. For each scholarship amount, predict students who will switch and attend

**Reminder Constraints**

fund_amount
average_student_sat
award_amounts
maximum_student_enrollment

In [499]:
df_IsEnrolled = df.IsEnrolledOrMatriculated
df_2 = df.copy()
#df_2.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)

In [500]:
#df_2.describe()

In [501]:
df_unscaled = pd.DataFrame(model_scalar.inverse_transform(df_2), columns=df_2.columns)
#df_unscaled['IsEnrolledOrMatriculated'] = df_IsEnrolled
#df_unscaled.describe()

In [502]:
df_unscaled.tuition_after_grants.describe()

count    15102.000000
mean     31604.371739
std       3219.878965
min          0.000000
25%      28643.000000
50%      34143.000000
75%      34143.000000
max      34143.000000
Name: tuition_after_grants, dtype: float64

In [503]:
df_unscaled['PuGo_Amount'] = 0
df_unscaled['EnrollPrediction'] = 0
df_unscaled['EnrollPredictionTemp'] = 0
df_unscaled['OriginalTuition_After_Grants'] = df_unscaled['tuition_after_grants']

In [504]:
print (df_unscaled.columns)

df_unscaled.columns[1:27]

Index(['IsEnrolledOrMatriculated', 'EFC', 'HighSchoolGPA', 'TestScore',
       'AcademicScore', 'tuition_after_grants', 'ed_0', 'ed_1', 'ed_2', 'ed_3',
       'ed_4', 'ed_5', 'ed_6', 'finaid_FIN1 - No FAFSA',
       'finaid_FIN2 - $0 Needed', 'finaid_FIN3 - $1 - $15K Needed',
       'finaid_FIN4 - $15K - $30K Needed', 'finaid_FIN5 - $30K - $40K Needed',
       'finaid_FIN6 - $40K+ Needed', 'acad_ACAD1 - 1106 SAT',
       'acad_ACAD2 - 1258 SAT', 'acad_ACAD3 - 1283 SAT',
       'acad_ACAD4 - 1297 SAT', 'acad_ACAD5 - 1314 SAT',
       'acad_ACAD6 - 1327 SAT', 'acad_ACAD7 - 1350 SAT',
       'acad_ACAD8 - 1451 SAT', 'PuGo_Amount', 'EnrollPrediction',
       'EnrollPredictionTemp', 'OriginalTuition_After_Grants'],
      dtype='object')


Index(['EFC', 'HighSchoolGPA', 'TestScore', 'AcademicScore',
       'tuition_after_grants', 'ed_0', 'ed_1', 'ed_2', 'ed_3', 'ed_4', 'ed_5',
       'ed_6', 'finaid_FIN1 - No FAFSA', 'finaid_FIN2 - $0 Needed',
       'finaid_FIN3 - $1 - $15K Needed', 'finaid_FIN4 - $15K - $30K Needed',
       'finaid_FIN5 - $30K - $40K Needed', 'finaid_FIN6 - $40K+ Needed',
       'acad_ACAD1 - 1106 SAT', 'acad_ACAD2 - 1258 SAT',
       'acad_ACAD3 - 1283 SAT', 'acad_ACAD4 - 1297 SAT',
       'acad_ACAD5 - 1314 SAT', 'acad_ACAD6 - 1327 SAT',
       'acad_ACAD7 - 1350 SAT', 'acad_ACAD8 - 1451 SAT'],
      dtype='object')

### Part C1 - Predict which students will attend with no PuGo scholarship

In [505]:
df_scaled = pd.DataFrame(model_scalar.transform(df_unscaled.ix[:,0:27]), columns=df_unscaled.columns[0:27])
df_scaled['IsEnrolledOrMatriculated'] = df_IsEnrolled
df_scaled.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)
df_scaled.columns

Index(['EFC', 'HighSchoolGPA', 'TestScore', 'AcademicScore',
       'tuition_after_grants', 'ed_0', 'ed_1', 'ed_2', 'ed_3', 'ed_4', 'ed_5',
       'ed_6', 'finaid_FIN1 - No FAFSA', 'finaid_FIN2 - $0 Needed',
       'finaid_FIN3 - $1 - $15K Needed', 'finaid_FIN4 - $15K - $30K Needed',
       'finaid_FIN5 - $30K - $40K Needed', 'finaid_FIN6 - $40K+ Needed',
       'acad_ACAD1 - 1106 SAT', 'acad_ACAD2 - 1258 SAT',
       'acad_ACAD3 - 1283 SAT', 'acad_ACAD4 - 1297 SAT',
       'acad_ACAD5 - 1314 SAT', 'acad_ACAD6 - 1327 SAT',
       'acad_ACAD7 - 1350 SAT', 'acad_ACAD8 - 1451 SAT'],
      dtype='object')

In [506]:
df_scaled.describe()

EFC  HighSchoolGPA     TestScore  AcademicScore  \
count  1.510200e+04   1.510200e+04  1.510200e+04   1.510200e+04   
mean   2.233017e-16  -2.862114e-15  1.406488e-16   8.290724e-16   
std    1.000033e+00   1.000033e+00  1.000033e+00   1.000033e+00   
min   -5.720723e-01  -1.166585e+01 -6.641407e+00  -1.209178e+01   
25%   -5.277606e-01  -3.992353e-01 -4.105461e-01  -3.972065e-01   
50%   -2.857465e-01   2.337204e-01  1.044011e-01   2.139645e-01   
75%    1.243007e-01   6.767894e-01  6.450956e-01   6.686339e-01   
max    1.098473e+01   9.932673e-01  1.597748e+00   1.280850e+00   

       tuition_after_grants          ed_0          ed_1          ed_2  \
count          1.510200e+04  1.510200e+04  1.510200e+04  1.510200e+04   
mean          -2.718989e-14  1.598509e-14 -7.883745e-16  9.259644e-15   
std            1.000033e+00  1.000033e+00  1.000033e+00  1.000033e+00   
min           -9.815716e+00 -1.567132e-01 -1.070172e-01 -3.171253e-01   
25%           -9.197457e-01 -1.567132e-01 -1.070172e-01 -3.171253e-01   
50%            7.884496e-01 -1.567132e-01 -1.070172e-01 -3.171253e-01   
75%            7.884496e-01 -1.567132e-01 -1.070172e-01 -3.171253e-01   
max            7.884496e-01  6.381084e+00  9.344292e+00  3.153328e+00   

               ed_3          ed_4          ...            \
count  1.510200e+04  1.510200e+04          ...             
mean   1.439486e-14  5.175729e-15          ...             
std    1.000033e+00  1.000033e+00          ...             
min   -9.579741e-01 -2.110517e-01          ...             
25%   -9.579741e-01 -2.110517e-01          ...             
50%   -9.579741e-01 -2.110517e-01          ...             
75%    1.043870e+00 -2.110517e-01          ...             
max    1.043870e+00  4.738176e+00          ...             

       finaid_FIN5 - $30K - $40K Needed  finaid_FIN6 - $40K+ Needed  \
count                      1.510200e+04                1.510200e+04   
mean                      -4.077441e-15               -1.994138e-15   
std                        1.000033e+00                1.000033e+00   
min                       -3.656483e-01               -6.097432e-01   
25%                       -3.656483e-01               -6.097432e-01   
50%                       -3.656483e-01               -6.097432e-01   
75%                       -3.656483e-01                1.640035e+00   
max                        2.734869e+00                1.640035e+00   

       acad_ACAD1 - 1106 SAT  acad_ACAD2 - 1258 SAT  acad_ACAD3 - 1283 SAT  \
count           1.510200e+04           1.510200e+04           1.510200e+04   
mean           -1.055346e-14           4.878277e-15          -1.755611e-16   
std             1.000033e+00           1.000033e+00           1.000033e+00   
min            -4.522149e-01          -3.570991e-01          -3.313433e-01   
25%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
50%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
75%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
max             2.211338e+00           2.800343e+00           3.018018e+00   

       acad_ACAD4 - 1297 SAT  acad_ACAD5 - 1314 SAT  acad_ACAD6 - 1327 SAT  \
count           1.510200e+04           1.510200e+04           1.510200e+04   
mean            6.836238e-15          -3.433708e-15           5.310067e-15   
std             1.000033e+00           1.000033e+00           1.000033e+00   
min            -3.911527e-01          -3.720200e-01          -3.659973e-01   
25%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
50%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
75%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
max             2.556546e+00           2.688027e+00           2.732260e+00   

       acad_ACAD7 - 1350 SAT  acad_ACAD8 - 1451 SAT  
count           1.510200e+04           1.510200e+04  
mean            6.571422e-15          -9.111541e-15  
std  

In [507]:
# Interaction terms
interactions_x = pd.DataFrame(preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(df_scaled))
# this produces a huge number of interaction terms. 

useful_interactions_x = interactions_x[useful_interaction_list]
#useful_interactions_x.columns
df_scaled = useful_interactions_x.join(df_scaled)
# Now describe the results
df_scaled.describe()

51           133           126            73            59  \
count  15102.000000  15102.000000  15102.000000  15102.000000  15102.000000   
mean       0.307604      0.342152     -0.282558      0.269323      0.148399   
std        1.949152      0.767162      1.000625      0.771365      0.687831   
min      -11.430328     -6.312629     -3.173904     -0.345497     -8.074341   
25%       -0.100908     -0.356549     -0.421363     -0.169364     -0.249364   
50%        0.067449      0.275473     -0.421363      0.006770     -0.029176   
75%        0.436160      0.696821      0.408540      0.303994      0.301107   
max       77.477632      4.438813      5.245716      4.057832      4.509156   

                EFC  HighSchoolGPA     TestScore  AcademicScore  \
count  1.510200e+04   1.510200e+04  1.510200e+04   1.510200e+04   
mean   2.233017e-16  -2.862114e-15  1.406488e-16   8.290724e-16   
std    1.000033e+00   1.000033e+00  1.000033e+00   1.000033e+00   
min   -5.720723e-01  -1.166585e+01 -6.641407e+00  -1.209178e+01   
25%   -5.277606e-01  -3.992353e-01 -4.105461e-01  -3.972065e-01   
50%   -2.857465e-01   2.337204e-01  1.044011e-01   2.139645e-01   
75%    1.243007e-01   6.767894e-01  6.450956e-01   6.686339e-01   
max    1.098473e+01   9.932673e-01  1.597748e+00   1.280850e+00   

       tuition_after_grants          ...            \
count          1.510200e+04          ...             
mean          -2.718989e-14          ...             
std            1.000033e+00          ...             
min           -9.815716e+00          ...             
25%           -9.197457e-01          ...             
50%            7.884496e-01          ...             
75%            7.884496e-01          ...             
max            7.884496e-01          ...             

       finaid_FIN5 - $30K - $40K Needed  finaid_FIN6 - $40K+ Needed  \
count                      1.510200e+04                1.510200e+04   
mean                      -4.077441e-15               -1.994138e-15   
std                        1.000033e+00                1.000033e+00   
min                       -3.656483e-01               -6.097432e-01   
25%                       -3.656483e-01               -6.097432e-01   
50%                       -3.656483e-01               -6.097432e-01   
75%                       -3.656483e-01                1.640035e+00   
max                        2.734869e+00                1.640035e+00   

       acad_ACAD1 - 1106 SAT  acad_ACAD2 - 1258 SAT  acad_ACAD3 - 1283 SAT  \
count           1.510200e+04           1.510200e+04           1.510200e+04   
mean           -1.055346e-14           4.878277e-15          -1.755611e-16   
std             1.000033e+00           1.000033e+00           1.000033e+00   
min            -4.522149e-01          -3.570991e-01          -3.313433e-01   
25%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
50%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
75%            -4.522149e-01          -3.570991e-01          -3.313433e-01   
max             2.211338e+00           2.800343e+00           3.018018e+00   

       acad_ACAD4 - 1297 SAT  acad_ACAD5 - 1314 SAT  acad_ACAD6 - 1327 SAT  \
count           1.510200e+04           1.510200e+04           1.510200e+04   
mean            6.836238e-15          -3.433708e-15           5.310067e-15   
std             1.000033e+00           1.000033e+00           1.000033e+00   
min            -3.911527e-01          -3.720200e-01          -3.659973e-01   
25%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
50%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
75%            -3.911527e-01          -3.720200e-01          -3.659973e-01   
max             2.556546e+00           2.688027e+00           2.732260e+00   

       acad_ACAD7 - 1350 SAT  acad_ACAD8 - 1451 SAT  
count           1.510200e+04           1.510200e+04  
mean            6.571422e-15          -9.111541e-15  
std   

In [508]:
# df_2.ix[:,0:31] <- use this to ignore columns added at the end
df_unscaled['EnrollPredictionTemp'] = predict_model.predict(df_scaled)

df_unscaled.describe()

IsEnrolledOrMatriculated            EFC  HighSchoolGPA     TestScore  \
count              15102.000000   15102.000000   1.510200e+04  15102.000000   
mean                   0.193993   49500.879023   3.686150e+00   1289.725864   
std                    0.123000   86531.922569   3.159883e-01    194.201115   
min                    0.143623       0.000000   4.440892e-16      0.000000   
25%                    0.143623    3834.250000   3.560000e+00   1210.000000   
50%                    0.143623   24775.500000   3.760000e+00   1310.000000   
75%                    0.143623   60256.500000   3.900000e+00   1415.000000   
max                    0.494331  999999.000000   4.000000e+00   1600.000000   

       AcademicScore  tuition_after_grants          ed_0          ed_1  \
count   15102.000000          15102.000000  15102.000000  15102.000000   
mean        0.904219          31604.371739      0.023970      0.011323   
std         0.074782           3219.878965      0.152962      0.105809   
min         0.000000              0.000000      0.000000      0.000000   
25%         0.874516          28643.000000      0.000000      0.000000   
50%         0.920219          34143.000000      0.000000      0.000000   
75%         0.954219          34143.000000      0.000000      0.000000   
max         1.000000          34143.000000      1.000000      1.000000   

               ed_2          ed_3              ...               \
count  15102.000000  15102.000000              ...                
mean       0.091379      0.478546              ...                
std        0.288156      0.499556              ...                
min        0.000000      0.000000              ...                
25%        0.000000      0.000000              ...                
50%        0.000000      0.000000              ...                
75%        0.000000      1.000000              ...                
max        1.000000      1.000000              ...                

       acad_ACAD3 - 1283 SAT  acad_ACAD4 - 1297 SAT  acad_ACAD5 - 1314 SAT  \
count           15102.000000           1.510200e+04           1.510200e+04   
mean                0.098927           1.326977e-01           1.215733e-01   
std                 0.298574           3.392589e-01           3.268032e-01   
min                 0.000000           2.775558e-17           1.387779e-17   
25%                 0.000000           2.775558e-17           1.387779e-17   
50%                 0.000000           2.775558e-17           1.387779e-17   
75%                 0.000000           2.775558e-17           1.387779e-17   
max                 1.000000           1.000000e+00           1.000000e+00   

       acad_ACAD6 - 1327 SAT  acad_ACAD7 - 1350 SAT  acad_ACAD8 - 1451 SAT  \
count           1.510200e+04           15102.000000           15102.000000   
mean            1.181300e-01               0.107933               0.137863   
std             3.227727e-01               0.310306               0.344767   
min            -2.775558e-17               0.000000               0.000000   
25%            -2.775558e-17               0.000000               0.000000   
50%            -2.775558e-17               0.000000               0.000000   
75%            -2.775558e-17               0.000000               0.000000   
max             1.000000e+00               1.000000               1.000000   

       PuGo_Amount  EnrollPrediction  EnrollPredictionTemp  \
count      15102.0           15102.0          15102.000000   
mean           0.0               0.0              0.046087   
std            0.0               0.0              0.209680   
min            0.0               0.0              0.000000   
25%            0.0               0.0              0.000000   
50%            0.0               0.0              0.000000   
75%            0.0               0.0              0.000000   
max            0.0               0.0              1.000000   

       OriginalTuition_After_Grants  
count                

In [509]:
df_unscaled['EnrollPrediction'] = df_unscaled['EnrollPredictionTemp']

In [510]:
print ('Students who will enroll with no PuGo funding:',df_unscaled.EnrollPrediction.sum())

Students who will enroll with no PuGo funding: 696


### Part C2 - For each scholarship amount, predict students who will switch and attend

In [511]:
for award_amount in sorted(award_amounts):
    df_unscaled.PuGo_Amount[df_unscaled['EnrollPrediction'] == 0] = award_amount
    #print (df_unscaled.PuGo_Amount.describe())
    df_unscaled['tuition_after_grants'] = df_unscaled['OriginalTuition_After_Grants'] - df_unscaled['PuGo_Amount']
    #print (df_unscaled.tuition_after_grants.describe())
    # Now scale back down    
    df_scaled = pd.DataFrame(model_scalar.transform(df_unscaled.ix[:,0:27]), columns=df_unscaled.columns[0:27])
    df_scaled.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)
    # Now add interaction terms
    interactions_x = pd.DataFrame(preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(df_scaled))
    useful_interactions_x = interactions_x[useful_interaction_list]
    df_scaled = useful_interactions_x.join(df_scaled)
    # Now predict
    df_unscaled['EnrollPredictionTemp'] = predict_model.predict(df_scaled)
    # Now print the results 
    print ('Scholarship amount',award_amount)
    print ('Old counts',np.unique(df_unscaled.EnrollPrediction, return_counts = True))
    print ('New counts',np.unique(df_unscaled.EnrollPredictionTemp, return_counts = True))    
    df_unscaled['EnrollPrediction'] = df_unscaled['EnrollPredictionTemp']

Scholarship amount 1000
Old counts (array([0, 1], dtype=int64), array([14406,   696], dtype=int64))
New counts (array([0, 1], dtype=int64), array([14330,   772], dtype=int64))


C:\Users\fractals.000\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Scholarship amount 2000
Old counts (array([0, 1], dtype=int64), array([14330,   772], dtype=int64))
New counts (array([0, 1], dtype=int64), array([14241,   861], dtype=int64))
Scholarship amount 3000
Old counts (array([0, 1], dtype=int64), array([14241,   861], dtype=int64))
New counts (array([0, 1], dtype=int64), array([14116,   986], dtype=int64))
Scholarship amount 4000
Old counts (array([0, 1], dtype=int64), array([14116,   986], dtype=int64))
New counts (array([0, 1], dtype=int64), array([13925,  1177], dtype=int64))
Scholarship amount 5000
Old counts (array([0, 1], dtype=int64), array([13925,  1177], dtype=int64))
New counts (array([0, 1], dtype=int64), array([13777,  1325], dtype=int64))
Scholarship amount 6000
Old counts (array([0, 1], dtype=int64), array([13777,  1325], dtype=int64))
New counts (array([0, 1], dtype=int64), array([13649,  1453], dtype=int64))
Scholarship amount 6500
Old counts (array([0, 1], dtype=int64), array([13649,  1453], dtype=int64))
New counts (array([0

## Part D - Calculate and visualize yield, net revenue, and scholarship info per segment.

In [512]:
predicted_pugo_offered = df_unscaled.PuGo_Amount.sum()
print ('PuGo Offered',predicted_pugo_offered)

predicted_pugo_use = df_unscaled.PuGo_Amount[df_unscaled['EnrollPrediction'] == 1].sum()
print ('PuGo Use',predicted_pugo_use)

PuGo Offered 91619500
PuGo Use 3525000


In [513]:
predicted_student_count = df_unscaled.EnrollPrediction.sum()
total_student_count = df_unscaled.EnrollPrediction.count()

print ('Predicted students:',predicted_student_count)
print ('Admitted students:',total_student_count)
print ('Predicted yield:',predicted_student_count * 100.0 / total_student_count)

Predicted students 1549
Total students 15102
Predicted yield 10.2569196133


In [514]:
predicted_net_revenue = df_unscaled.tuition_after_grants[df_unscaled['EnrollPrediction'] == 1].sum()
print ('Predicted Net Revenue:',predicted_net_revenue)

Net Revenue 44289281.0


In [515]:
predicted_avg_sat = df_unscaled.TestScore[df_unscaled['EnrollPrediction'] == 1].mean()
print ('Predicted Average SAT:',predicted_avg_sat)

Predicted Average SAT 1260.8037443511944


In [516]:
#df_unscaled.tuition_after_grants.describe()

In [517]:
df_unscaled.to_csv('Predictions.csv', header=True, index=False)

## Part E - Pass 2, Transfer Learning